In [ ]:
import requests
from bs4 import BeautifulSoup
from googletrans import Translator # Translator 클래스 임포트

def get_webpage_content(url):
    """
    주어진 URL의 웹 페이지 내용을 가져옵니다.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지를 가져오는 중 오류 발생: {e}") # f-string 형식으로 수정
        return None

def parse_quotes(html_content):
    """
    HTML 내용에서 명언과 저자를 파싱하여 리스트 형태로 반환합니다.
    """
    if not html_content:
        return []

    soup = BeautifulSoup(html_content, 'html.parser')
    quotes_data = []
    quote_blocks = soup.find_all('div', class_='quote') 

    for quote_block in quote_blocks:
        text_tag = quote_block.find('span', class_='text')
        author_tag = quote_block.find('small', class_='author')

        if text_tag and author_tag:
            quote_text = text_tag.get_text(strip=True)
            author_name = author_tag.get_text(strip=True)
            quotes_data.append({'quote': quote_text, 'author': author_name})
    return quotes_data

def translate_text(text, dest_lang='ko'):
    """
    주어진 텍스트를 지정된 언어로 번역합니다. (기본값: 한국어)
    """
    translator = Translator()
    try:
        # 번역 요청 시 지연 시간을 두어 너무 많은 요청으로 인한 차단 방지
        # 실제 사용 시 이 부분을 좀 더 세밀하게 조절해야 할 수 있습니다.
        # time.sleep(0.1) # 짧은 지연 시간 추가 (옵션)
        translated = translator.translate(text, dest=dest_lang)
        return translated.text
    except Exception as e:
        print(f"텍스트 번역 중 오류 발생: {e}")
        return text # 번역 실패 시 원본 텍스트 반환

if __name__ == "__main__":
    target_url = "http://quotes.toscrape.com/"
    print(f"대상 URL: {target_url}")

    html_content = get_webpage_content(target_url)

    if html_content:
        print("\n--- 명언 파싱 시작 ----")
        extracted_quotes = parse_quotes(html_content)

        if extracted_quotes:
            print(f"총 {len(extracted_quotes)}개의 명언을 찾았습니다.")
            print("\n--- 번역된 명언 (첫 5개) ---") # 메시지 변경

            for i, item in enumerate(extracted_quotes[:5]):
                original_quote = item['quote']
                author = item['author']

                # 명언 번역
                translated_quote = translate_text(original_quote, dest_lang='ko')

                print(f"[{i+1}] 원본: \"{original_quote}\"")
                print(f"      번역: \"{translated_quote}\" - {author}") # 저자는 번역하지 않고 그대로 출력
            print("-------------------------\n")
        else:
            print("명언을 찾지 못했습니다. 선택자를 확인하세요.")
    else:
        print("HTML 내용을 가져오지 못했습니다.")

대상 URL: http://quotes.toscrape.com/

--- 명언 파싱 시작 ----
총 10개의 명언을 찾았습니다.

--- 번역된 명언 (첫 5개) ---
[1] 원본: "“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”"
      번역: ""우리가 창조 한 세상은 우리의 생각의 과정입니다. 우리의 생각을 바꾸지 않고는 변할 수 없습니다."" - Albert Einstein
[2] 원본: "“It is our choices, Harry, that show what we truly are, far more than our abilities.”"
      번역: ""해리, 우리의 선택은 우리의 능력보다 훨씬 더 많은 것을 보여주는 것이 우리의 선택입니다."" - J.K. Rowling
[3] 원본: "“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”"
      번역: ""당신의 삶을 살 수있는 방법은 두 가지뿐입니다. 한 사람은 아무것도 기적이 아닙니다. 다른 하나는 모든 것이 기적 인 것처럼 보입니다."" - Albert Einstein
[4] 원본: "“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”"
      번역: ""좋은 소설에서 즐거움을주지 않는 신사 나 여인이라면 참을 수 없을 정도로 멍청해야합니다."" - Jane Austen
[5] 원본: "“Imperfection is beauty, madness is genius and it's better to b